In [1]:
from utils import *
from model_prophet import *

/Users/julius/repos/AFC/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [ ]:
data = Data()
model = ProphetModel(data.merged_data)

In [ ]:
model.generate_forecasts()

In [ ]:
model.save_forecasts_to_csv('submission_prophet_model.csv')
model.df_all_forecasts

In [ ]:
rmse = model.compute_rmse(data.sales_test)
print(f'RMSE: {rmse}')